# Conjugated clifford experiment

Define clifford conjugation cicuit as

$$
U = H^{\otimes n} C^\dagger P C H^{\otimes n}
$$

where $C \in \text{Cl}(2^n)$ is a clifford circuit over $n$ qubits, $P = P_{p_1} \otimes \dots \otimes P_{p_n}$ is an $n$-local Pauli string with each $p_k \in \{I, X, Y, Z\}$. By stabilizer formalism, since $H \in \text{Cl}(2)$ we have that the action of $U$ simplifies to a Pauli operator acting on a computational basis state, and so the output of this circuit is uniqely described by a computational basis bitstring that we can easily simulate.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random

import cirq
import cirq_google as cg
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx 

from qbitmap import metrics
from qbitmap import utils
from qbitmap import calibration_data
from qbitmap.noise.simple_noise_model import SimpleNoiseModel
from qbitmap import circuits
from qbitmap import diagnostics

PROJECT ID: fermilab-quantum
PROCESSOR:  >>> rainbow <<<
GATESET:    <cirq_google.serializable_gate_set.SerializableGateSet object at 0x7fb65c05b160>


In [ ]:
from qbitmap import hw

# Don't overwrite any of the analysis using a different processor...!
assert hw.PROCESSOR_ID == "rainbow"


# target_calibration = hw.PROCESSOR.get_current_calibration()
# TIMESTAMP = target_calibration.timestamp 
TIMESTAMP = 1630400395629
target_calibration = hw.PROCESSOR.get_calibration(TIMESTAMP)

print("TIMESTAMP:", TIMESTAMP)

# Target specifically these metrics
metric_1q = "single_qubit_rb_average_error_per_gate"
metric_2q = "two_qubit_sqrt_iswap_gate_xeb_average_error_per_cycle"
calibration = calibration_data.CalibrationWrapper(
    calibration_dct=target_calibration, 
    qubits=hw.DEVICE.qubits, 
    metric_1q=metric_1q, 
    metric_2q=metric_2q
)
fig, ax = plt.subplots(figsize=(10, 10))

calibration.plot_noise_graph(ax=ax)
print()

## Hardware submissions


In [21]:
def pseudo_random_clifford(qubits, c=None, twoq_density=0.4, seed=None):
    """Generate a random clifford circuit, lazily.
    
    Disclaimer: This is _not_ random in the sense that it returns
    a clifford operation sampled uniformly with respect to the
    clifford group.
    
    I have tried to at least guaranteed that the circuit has nontrivial
    entanglement properties
    
    Optionally provide a circuit `c` to build upon
    
    Args:
        
    """
    if seed:
        np.random.seed(seed)
    if c is None:
        c = cirq.Circuit()
    ops_1q = [cirq.H, cirq.S, cirq.X, cirq.Y]
    ops_2q = [cirq.CNOT]
    
    pairs_to_hit = [(qubits[i], qubits[i+1]) for i in range(len(qubits) - 1)]
    np.random.shuffle(pairs_to_hit)
    while len(pairs_to_hit) > 0:
        
        coinflip = np.random.random()
    
        if coinflip < twoq_density:
            # Apply a two-qubit gate
            op = np.random.choice(ops_2q)
            c += op.on(*pairs_to_hit.pop())
        else:
            # Enforce that we do not duplicate any single-qubit gates
            src = np.random.choice(range(len(qubits)))
            ops_on_q = list(c.findall_operations(predicate=lambda x: qubits[src] in x.qubits))
            op = np.random.choice(ops_1q)
            temp = [x for x in ops_1q]
            if any(ops_on_q):
                _, last_op =ops_on_q[-1]
                if last_op.gate in temp:
                    temp.remove(last_op.gate)
                    
            op = np.random.choice(temp)
            c += op.on(qubits[src])
    return c


In [22]:
def random_pauli_circuit(qubits):
    """A random pauli string, in circuit form."""
    out = cirq.Circuit()
    ops = [cirq.X, cirq.Y, cirq.Z]
    for q in qubits:
        out += np.random.choice(ops).on(q)
    return out

def random_cl2_by_n(qubits):
    c = cirq.Circuit()
    for q in qubits:
        basis_idx = np.random.randint(3)
        if basis_idx == 0: # Z-basis
            continue
        elif basis_idx == 1: # X-basis
            c += cirq.H.on(q)
        elif basis_idx == 2: # Y-basis
            c += cirq.inverse(cirq.S).on(q)
            c += cirq.H.on(q)
    return c

def make_rigged_clif(qubits, seed=None):
#     qubits = cirq.LineQubit.range(n)
#     S = random_cl2_by_n(qubits)
    S = cirq.Circuit(cirq.H.on_each(qubits))
    U = pseudo_random_clifford(qubits, c=S, seed=seed)
    P = random_pauli_circuit(qubits)
    full = cirq.Circuit()
#     full.append(S, strategy=cirq.InsertStrategy.EARLIEST)
    full.append(U, strategy=cirq.InsertStrategy.EARLIEST)
    full.append(P, strategy=cirq.InsertStrategy.EARLIEST)
    full.append(cirq.inverse(U), strategy=cirq.InsertStrategy.EARLIEST)
#     full.append(cirq.inverse(S), strategy=cirq.InsertStrategy.EARLIEST)
    return full
template_qubits = cirq.GridQubit.rect(1, 8)
n_qubits = len(template_qubits)

clifford = make_rigged_clif(template_qubits, seed=7)
structured_circuit = cg.optimized_for_sycamore(clifford)
structured_circuit_with_measure = structured_circuit + cirq.measure(*template_qubits, key='m')
forward_depth = len(structured_circuit)

clifford_LE = clifford + cirq.inverse(clifford) + cirq.measure(*template_qubits, key='m')
# Concatenate for LE circuit _after_ compilation to hardware gates
structured_circuit_LE = cg.optimized_for_sycamore(clifford) + cg.optimized_for_sycamore(cirq.inverse(clifford)) + cirq.measure(*template_qubits, key='m')

print("FORWARDS LOGICAL CIRCUIT")
display(clifford)
print("FORWARDS LOGICAL DEPTH=", len(clifford))
display(clifford_LE)

print("FORWARDS CIRCUIT")
display(structured_circuit)
print("FORWARDS DEPTH=", forward_depth)
display(structured_circuit_LE)
print("LE DEPTH=", len(structured_circuit_LE))

# Precompute the correct final bitstring hr
wf = clifford.final_state_vector()
atol = 1e-6
loc = np.where(abs(abs(wf) - 1) < atol)[0][0]
print("Expect bitstring with idx {}".format(loc))

FORWARDS LOGICAL CIRCUIT


(0, 0): ───H───Y───X───S───Y───X───@───────X───────@───X───Y───S^-1───X───Y──────H───
                                   │               │
(0, 1): ───H───@───────────────────X───────Z───────X──────────────────────@──────H───
               │                                                          │
(0, 2): ───H───X───Y───X───────────────@───Z───@───────────────X──────Y───X──────H───
                                       │       │
(0, 3): ───H───────@───Y───H───X───Y───X───Y───X───Y───X───H───Y──────@──────────H───
                   │                                                  │
(0, 4): ───H───@───X───S───H───X───────────Z───────────X───H───S^-1───X───@──────H───
               │                                                          │
(0, 5): ───H───X───X───S───Y───H───X───@───Y───@───X───H───Y───S^-1───X───X──────H───
                                       │       │
(0, 6): ───H───S───────@───────────────X───X───X───────────────@──────────S^-1───H───
                       │                                       │
(0, 7): ───H───S───H───X───────────────────Y───────────────────X──────H───S^-1───H───

FORWARDS LOGICAL DEPTH= 17


(0, 0): ───H───Y───X───S───Y───X───@───────X───────@───X───Y───S^-1───X───Y──────H───H───Y───X───S───Y───X───@───────X───────@───X───Y───S^-1───X───Y──────H───M('m')───
                                   │               │                                                         │               │                                 │
(0, 1): ───H───@───────────────────X───────Z───────X──────────────────────@──────H───H───@───────────────────X───────Z───────X──────────────────────@──────H───M────────
               │                                                          │              │                                                          │          │
(0, 2): ───H───X───Y───X───────────────@───Z───@───────────────X──────Y───X──────H───H───X───Y───X───────────────@───Z───@───────────────X──────Y───X──────H───M────────
                                       │       │                                                                 │       │                                     │
(0, 3): ───H───────@───Y───H───X───Y───X───Y───X───Y───X───H───Y──────@──────────H───H───────@───Y───H───X───Y───X───Y───X───Y───X───H───Y──────@──────────H───M────────
                   │                                                  │                      │                                                  │              │
(0, 4): ───H───@───X───S───H───X───────────Z───────────X───H───S^-1───X───@──────H───H───@───X───S───H───X───────────Z───────────X───H───S^-1───X───@──────H───M────────
               │                                                          │              │                                                          │          │
(0, 5): ───H───X───X───S───Y───H───X───@───Y───@───X───H───Y───S^-1───X───X──────H───H───X───X───S───Y───H───X───@───Y───@───X───H───Y───S^-1───X───X──────H───M────────
                                       │       │                                                                 │       │                                     │
(0, 6): ───H───S───────@───────────────X───X───X───────────────@──────────S^-1───H───H───S───────@───────────────X───X───X───────────────@──────────S^-1───H───M────────
                       │                                       │                                 │                                       │                     │
(0, 7): ───H───S───H───X───────────────────Y───────────────────X──────H───S^-1───H───H───S───H───X───────────────────Y───────────────────X──────H───S^-1───H───M────────

FORWARDS CIRCUIT


(0, 0): ───PhXZ(a=0,x=0.5,z=-0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=0)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0.25,x=1,z=0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                               │                                 │                                     │                                 │
(0, 1): ───PhXZ(a=-0.75,x=0,z=-0.5)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0.5,x=0.5,z=-1)────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=-0.5)─────────────────────────────────────────────────────────────────────────────
                                       │                                 │                                                                                                                                                                                                                                                              │                                 │
(0, 2): ───PhXZ(a=-0.5,x=0.5,z=-0.5)───iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=-1,x=0.5,z=0.5)────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0.5,x=0.5,z=-1)─────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=-0.5)────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=-1)──────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                       │                                 │                                      │                                 │
(0, 3): ───PhXZ(a=-0.75,x=0,z=-0.5)────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0.5,x=0.5,z=0)─────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)───iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=-0.5)─────────────────────────────────────────────────────────────────────────────
                                                                                                               │                                 │                                                                                                                                                                                      │                                 │
(0, 4): ───PhXZ(a=-0.75,x=0,z=-0.5)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=-0.5,x=0.5,z=-1)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)───────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=-0.5)───
                                       │                                 │                                                                                                                                                                                                                                                                                                              

FORWARDS DEPTH= 25


(0, 0): ───PhXZ(a=0,x=0.5,z=-0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=0)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0.25,x=1,z=0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────PhXZ(a=0,x=0.5,z=-0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=0)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0.25,x=1,z=0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M('m')───
                                                                                                               │                                 │                                     │                                 │                                                                                                                                                                                                                                                                                                                                                                      │                                 │                                     │                                 │                                                                                                                                                                                                                                                                  │
(0, 1): ───PhXZ(a=-0.75,x=0,z=-0.5)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0.5,x=0.5,z=-1)────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=-0.5)─────────────────────────────────────────────────────────────────────────────PhXZ(a=-0.75,x=0,z=-0.5)────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=0,x=0.5,z=0.5)─────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0.5,x=0.5,z=-1)────iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=0,x=0.5,z=0.5)──────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=0)───iSwap───────PhXZ(a=-0.5,x=0.5,z=-0.5)─────────────────────────────────────────────────────────────────────────────M────────
                                       │                                 │                                                                                                                                                                                                                                                              │                                 │                                                                                                                                             │                                 │                                                                                                                                                                                                                                                              │                                 │                                                                                                                 │
(0, 2): ───PhXZ(a=-0.5,x=0.5,z=-0.5)───iSwap^-0.5────────────────────────iSwap^0.5───PhXZ(a=-1,x=0.5,z=0.5)────────────────────────────────────────────────────────────────────────────iSwap────────PhXZ(a=-1,x=1,z=

LE DEPTH= 51
Expect bitstring with idx 74


In [23]:
random_circuit_LE_half = circuits.loschmidt_circuit(
    circuits.create_random_line_circuit, 
    template_qubits, 
    depth=forward_depth // 2, 
    seed=123, 
    measure="m")
print("RANDOM CIRCUIT, depth={} (forward depth)".format(len(random_circuit_LE_half)))
display(random_circuit_LE_half)


RANDOM CIRCUIT, depth=25 (forward depth)


(0, 0): ───PhX(0.5)^0.5──────────────────────PhX(-0.25)^0.5───FSim(0.25π, 0)───PhX(0.75)^0.5─────────────────────PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(0)^0.5───────────────────────PhX(0.75)^0.5────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0.75)^-0.5──────────────────────PhX(0)^-0.5──────FSim(-0.25π, 0)───PhX(0.5)^-0.5───────────────────────PhX(0.75)^-0.5────FSim(-0.25π, 0)───PhX(-0.25)^-0.5─────────────────────PhX(0.5)^-0.5─────M────────
                                                              │                                                                   │                                                                  │                │                                                                      │                                                                       │                                                                       │
(0, 1): ───PhX(0.25)^0.5────FSim(0.25π, 0)───PhX(-0.5)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.25)^0.5────FSim(0.25π, 0)───PhX(0.75)^0.5───FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(0.75)^-0.5───FSim(-0.25π, 0)───PhX(0.25)^-0.5────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5────FSim(-0.25π, 0)───PhX(0.25)^-0.5────M────────
                            │                                                                   │                                                                  │                                                                                      │                                                                      │                                                                       │                                   │
(0, 2): ───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(-0.25)^0.5───FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(0)^0.5───────FSim(0.25π, 0)───PhX(-0.5)^0.5───FSim(0.25π, 0)───PhX(-0.25)^0.5───FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(-0.25)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5───FSim(-0.25π, 0)───PhX(0)^-0.5───────FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(-0.25)^-0.5───FSim(-0.25π, 0)───PhX(0.5)^-0.5─────M('m')───
                                                              │                                                                   │                                                                  │                │                                                                      │                                                                       │                                                                       │
(0, 3): ───PhX(-0.5)^0.5────FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(0)^0.5──────FSim(0.25π, 0)───PhX(0.5)^0.5─────FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(0.5)^-0.5─────FSim(-0.25π, 0)───PhX(0)^-0.5──────FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(0.5)^-0.5─────FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(-0.5)^-0.5────M────────
                            │                                                                   │                                                                  │                                                                                      │                                                                      │                                                                       │                                   │
(0, 4): ───PhX(0)^0.5───────FSim(0.25π, 0)───PhX(0.5)^0.5─────FSim(0.25π, 0)───PhX(1)^0.5───────FSim(0.25π, 0)───PhX(0)^0.5───────FSim(0.25π, 0)───PhX(0.75)^0.5───FSim(0.25π, 0)───PhX(-0.75)^0.5───FSim(0.25π, 0)───FSim(-0.25π, 0)───PhX(-0.75)^-0.5───FSim(-0.25π, 0)───PhX(0.75)^-0.5───FSim(-0.25π, 0)───PhX(0)^-0.5───────FSim(-0.25π, 0)───PhX(1)^-0.5───────FSim(-0.25π, 0)───PhX(0.5)^-0.5─────FSim(-0.25π, 0)───PhX(0)^-0.5───────M────────
                                                              │                  

In [24]:
noise_graph = calibration.noise_graph
# Pre-compute all of the paths
all_paths = []
for source in noise_graph.nodes():
    for target in noise_graph.nodes():
        if source == target:
            continue            
        all_paths_ij = list(nx.all_simple_paths(noise_graph, source, target, cutoff=n_qubits))
        for v in all_paths_ij:
            if len(v) == n_qubits:
                all_paths.append(v)

print("Found {} paths with {} qubits to attempt".format(len(all_paths), n_qubits))

# Shuffle paths
shuffled_paths = [x for x in all_paths]
np.random.seed(49213)
random.shuffle(shuffled_paths)

Found 2984 paths with 8 qubits to attempt


### EXPERIMENT COMPLETED: DO NOT RUN


In [27]:
 # Dry run tests: 5 passes finished in 5:05 minutes
# = ~1 minute per full experiment


# Set DRY_RUN to dump readout ec metadata into a different folder
DRY_RUN = False
def dry_run_print(*s, dry_run=DRY_RUN):
    if dry_run:
        print(*s)
DATESTR = "20210901"

bf_timestamp = str(TIMESTAMP)
readout_ec_path = "./readout_ec"
if DRY_RUN:
    readout_ec_path = "./dryrun"
    bf_timestamp = str(TIMESTAMP) + str(np.random.random() * 1000)
    
DIAGNOSTIC_REPS = 10_000
REPETITIONS = 15_000
N_EXPERIMENTS = 350

# results[:,0] stores F_LE, results[:,1] stores F, results[:,2] stores F0
# Raw refers to no readout error correction, corr refers to yes readout EC
results_raw = np.zeros((N_EXPERIMENTS, 3))
results_corr = np.zeros((N_EXPERIMENTS, 3))

# Configuration for random circuits
N_TRIALS = 5 # Number of random circuits to attempt per qubit configuration
# For each random circuit, we'll attempt one with(forward depth) and one for 2*(forward depth)
random_results_raw = np.zeros((N_EXPERIMENTS, N_TRIALS))
random_results_corr = np.zeros((N_EXPERIMENTS, N_TRIALS))
seeds = [3882, 175, 802, 993, 22]
assert len(seeds) >= N_TRIALS

used_paths = []

for i in range(N_EXPERIMENTS):

    if (i % 5) == 0:
        print(f"run={i}")
        
    # Construct the circuit on this path
    v = shuffled_paths[i]
    targets = [cirq.GridQubit(*x) for x in v]
    qubit_map = dict(zip(template_qubits, targets))   
    used_paths.append(v)
    
    # Perform separable readout error diagnostic
    # Every iteration needs a unique identifier for its readout error diagnostic
    hw_diagnostic_sep = diagnostics.SeparableReadoutErrorDiagnostic(
        timestamp=bf_timestamp + f"_{i}" ,
        qubits=targets,
        repetitions=DIAGNOSTIC_REPS,
        debug=False,
        path=readout_ec_path
    )
    qvals = hw_diagnostic_sep.run(ntrials=1)
    dry_run_print("...readout error diagnostic complete")

#     Compute structured circuit loschmidt survival
#     Don't overwrite the structured LE circuit
    mapped_LE_circuit = structured_circuit_LE.transform_qubits(qubit_map)
    job = hw.ENGINE.run_sweep(
        program=mapped_LE_circuit,
        repetitions=REPETITIONS,
        processor_ids=[hw.PROCESSOR_ID],
        gate_set=hw.GATESET
    )
    # F_LE computation
    LE_counter = job.results()[0].histogram(key="m")
    results_raw[i,0] = LE_counter.get(0) / REPETITIONS
    LE_arr = utils.hist_as_np(LE_counter, n_qubits, REPETITIONS)
    results_corr[i,0] = hw_diagnostic_sep.invert_and_correct(LE_arr)[0]
    dry_run_print("...F_LE complete, FLE={}".format(results_corr[i,0]))

    # Fidelity of this clifford operation is just like FLE
    mapped_forward_circuit = structured_circuit_with_measure.transform_qubits(qubit_map)
    job = hw.ENGINE.run_sweep(
        program=mapped_forward_circuit,
        repetitions=REPETITIONS,
        processor_ids=[hw.PROCESSOR_ID],
        gate_set=hw.GATESET
    )
    # DFE computation
    DFE_counter = job.results()[0].histogram(key="m")
    results_raw[i,1] = DFE_counter.get(loc) / REPETITIONS
    DFE_arr = utils.hist_as_np(DFE_counter, n_qubits, REPETITIONS)
    results_corr[i,1] = hw_diagnostic_sep.invert_and_correct(DFE_arr)[loc]
    dry_run_print("...F DFE complete, F={}".format(results_corr[i,1]))

    # Perform random circuit runs with readout error correction
    rand_fwd_depth = forward_depth // 2 
    # iterate over random circuit depths
    for j in range(N_TRIALS):
        # Iterate over rand
        random_circuit_LE = circuits.loschmidt_circuit(
            circuits.create_random_line_circuit, 
            template_qubits, 
            depth=rand_fwd_depth, 
            seed=seeds[j] + i, 
            measure="m")
        dry_run_print("rand circuit LE depth={}".format(len(random_circuit_LE)))
        random_circuit_LE = random_circuit_LE.transform_qubits(qubit_map)
        # Submit hardware random circuit
        job = hw.ENGINE.run_sweep(
            program=random_circuit_LE,
            repetitions=REPETITIONS,
            processor_ids=[hw.PROCESSOR_ID],
            gate_set=hw.GATESET
        )
        # raw and corr
        random_counter = job.results()[0].histogram(key="m")
        random_results_raw[i, j] = random_counter.get(0) / REPETITIONS
        rand_arr = utils.hist_as_np(random_counter, n_qubits, REPETITIONS)
        random_results_corr[i,j] = hw_diagnostic_sep.invert_and_correct(rand_arr)[0]
    dry_run_print("...Random F_LE complete")
    dry_run_print(random_results_corr[i])

        
    # Perform zeroth order metric with and without readout error correction assumption
    F0 = metrics.compute_calibration_fidelity(mapped_forward_circuit, noise_graph)
    F0_readout_err = metrics.compute_calibration_fidelity(mapped_forward_circuit, noise_graph, readout_error=True)
    results_corr[i,2] = F0
    results_raw[i,2] = F0_readout_err
    
    if DRY_RUN and i == 25:
        break
#     Caching
    np.save(f"./temp/v1_hw_{DATESTR}_clifford_line_results_corr_{i}.npy", results_corr)
    np.save(f"./temp/v1_hw_{DATESTR}_clifford_line_results_raw_{i}.npy", results_raw)
    np.save(f"./temp/v1_hw_{DATESTR}_random_line_results_raw_{i}.npy", random_results_raw)
    np.save(f"./temp/v1_hw_{DATESTR}_random_line_results_corr_{i}.npy", random_results_corr)
    np.save(f"./temp/v1_hw_{DATESTR}_clifford_line_vs_random_paths_{i}.npy", np.asarray(used_paths))

if not DRY_RUN:
    np.save(f"v1_hw_{DATESTR}_clifford_line_results_corr.npy", results_corr)
    np.save(f"v1_hw_{DATESTR}_clifford_line_results_raw.npy", results_raw)
    np.save(f"v1_hw_{DATESTR}_random_line_results_raw.npy", random_results_raw)
    np.save(f"v1_hw_{DATESTR}_random_line_results_corr.npy", random_results_corr)
    np.save(f"v1_hw_{DATESTR}_clifford_line_vs_random_paths.npy", np.asarray(used_paths))

run=0
run=5
run=10
run=15
run=20
run=25
run=30
run=35
run=40
run=45
run=50
run=55
run=60
run=65
run=70
run=75
run=80
run=85
run=90
run=95
run=100
run=105
run=110
run=115
run=120
run=125
run=130
run=135
run=140
run=145
run=150
run=155
run=160
run=165
run=170
run=175
run=180
run=185
run=190
run=195
run=200
run=205
run=210
run=215
run=220
run=225
run=230
run=235
run=240
run=245
run=250
run=255
run=260
run=265
run=270
run=275
run=280
run=285
run=290
run=295
run=300
run=305
run=310
run=315
run=320
run=325
run=330
run=335
run=340
run=345
